# Control Flow

Seho Jeong, Sogang University

**References**
- **Perla, Jesse, Thomas J. Sargent, and John Stachurski. n.d.** "Quantitative Economics with Julia." QuantEcon.

## Iteration

One of the most important tasks in computing is stepping through a sequence of data and performing a given action. Julia provides neat and flexible tools for iteration as we now discuss.

### Iterables

An iterable is something you can put on the right hand side of `for` and loop over. These include sequence data types like arrays.

In [2]:
actions = ["surf", "ski"]
for action in actions
    println("Charlie doesn't $action.")
end

Charlie doesn't surf.
Charlie doesn't ski.


They also include so-called iterators.

In [3]:
for i in 1:3 
    print(i)
end

123

If you ask for the keys of dictionary you get an iterator.

In [4]:
d = Dict("name" => "Frodo", "age" => 33)

Dict{String, Any} with 2 entries:
  "name" => "Frodo"
  "age"  => 33

In [5]:
keys(d)

KeySet for a Dict{String, Any} with 2 entries. Keys:
  "name"
  "age"

This makes sense, since the most common thing you want to do with keys is loop over them. The benefit of providing an iterator rather than an array, say, is that the former is more memory efficient. Should you need to transform an iterator into an array you can always use `collect()`.

In [6]:
collect(keys(d))

2-element Vector{String}:
 "name"
 "age"

### Looping without Indicies

You can loop over sequences without explicit indexing, which often leads to neater code. For example, compare

In [10]:
x_values = 1:5;

In [11]:
for x in x_values
    println(x * x)
end

1
4
9
16
25


In [12]:
for i in eachindex(x_values) 
    println(x_values[i] * x_values[i])
end

1
4
9
16
25


Julia provides some functional-style helper functions (similar to Python and R) to facilitate looping without indicies. One is `zip()`, which is used for stepping through pairs from two sequences. For example, try running the following code.

In [13]:
countries = ("Japan", "Korea", "China")
cities = ("Tokyo", "Seoul", "Beijing")
for (country, city) in zip(countries, cities)
    println("The capital of $country is $city.")
end

The capital of Japan is Tokyo.
The capital of Korea is Seoul.
The capital of China is Beijing.


If we happen to need the index as well as the value, one option is to use `enumerate()`. The following snippet will give you the idea.

In [15]:
countries = ("Japan", "Korea", "China")
cities = ("Tokyo", "Seoul", "Beijing")
for (i, country) in enumerate(countries)
    city = cities[i]
    println("The capital of $country is $city.")
end

The capital of Japan is Tokyo.
The capital of Korea is Seoul.
The capital of China is Beijing.


### Comprehensions

Comprehensions are an elegant tool for creating new arrays, dictionaries, etc., from iterables. Here are some examples.

In [16]:
doubles = [2i for i in 1:4]

4-element Vector{Int64}:
 2
 4
 6
 8

In [17]:
animals = ["dog" , "cat", "bird"];

In [18]:
plurals = [animal * "s" for animal in animals]

3-element Vector{String}:
 "dogs"
 "cats"
 "birds"

In [19]:
[i + j for i in 1:3, j in 4:6]

3×3 Matrix{Int64}:
 5  6  7
 6  7  8
 7  8  9

In [20]:
[i + j + k for i in 1:3, j in 4:6, k in 7:9]

3×3×3 Array{Int64, 3}:
[:, :, 1] =
 12  13  14
 13  14  15
 14  15  16

[:, :, 2] =
 13  14  15
 14  15  16
 15  16  17

[:, :, 3] =
 14  15  16
 15  16  17
 16  17  18

Comprehensions can also create arrays of tuples or named tuples.

In [21]:
[(i, j) for i in 1:2, j in animals]

2×3 Matrix{Tuple{Int64, String}}:
 (1, "dog")  (1, "cat")  (1, "bird")
 (2, "dog")  (2, "cat")  (2, "bird")

In [22]:
[(num = i, animal = j) for i in 1:2, j in animals]

2×3 Matrix{@NamedTuple{num::Int64, animal::String}}:
 (num = 1, animal = "dog")  …  (num = 1, animal = "bird")
 (num = 2, animal = "dog")     (num = 2, animal = "bird")

Given two numeric arrays or tuples `x_vals` and `y_vals` of equal length, compute their inner product using `zip()`.

In [ ]:
x_vals = 1, 2, 3, 4
y_vals = 5, 6, 7, 8

result = sum(x * y for (x, y) in zip(x_vals, y_vals))

println("Sum = $result")

Sum = 70


Using a comprehension, count the number of even numbers between 0 and 99. (*Hint.* `iseven` returns `true` for even numbers and `false` for odds.)

In [12]:
count(iseven, 0:99)

50

In [7]:
sum(iseven(i) ? true : false for i in 0:99)

50

Using a comprehension, take `my_pairs = ((2, 5), (4, 2), (9, 8), (12, 10))` and count the number of pairs `(a, b)` such that both `a` and `b` are even.

In [15]:
my_pairs = ((2, 5), (4, 2), (9, 8), (12, 10))
count(x -> all(iseven, x), my_pairs)

2

### Generators

In some cases, you may wish to use a comprehension to create a iterable list rather than actually making it a concrete way. The benefit of this is that you can use functions which take general iterators rather than arrays without allocating and storing any temporary values. For example, the following code generates a temporary array of size 10,000 and finds the sum.

In [24]:
xs = 1:10000
f(x) = x^2
f_x = f.(xs)
sum(f_x)

333383335000

We could have created the temporary using a comprehension, or even done the comprehension within the `sum` function, but these all create temporary arrays.

In [26]:
f_x2 = [f(x) for x in xs]
@show sum(f_x2)
@show sum([f(x) for x in xs]);

sum(f_x2) = 333383335000
sum([f(x) for x = xs]) = 333383335000


Note that, if you were hand-code his, you would be able to calculate the sum by simply iterating to 10,000, applying `f` to each number, and accumulating the results. No temporary vectors would be necessary. A generator can emulate this behavior, leading to clear (and sometimes more efficient) code when used with any function that accepts iterators. All you need to do is drop the square brackets.

In [27]:
sum(f(x) for x in xs)

333383335000

We can use `BenchmarkTools` to investigate.

In [34]:
using BenchmarkTools

@btime sum([f(x) for x in $xs])
@btime sum(f.($xs))
@btime sum(f(x) for x in $xs);

  1.736 μs (3 allocations: 78.19 KiB)
  1.633 μs (3 allocations: 78.19 KiB)
  1.917 ns (0 allocations: 0 bytes)


Notice that the first two cases are nearly identical, and allocate a temporary array, while the final case using generators has no allocations. 

In this example you may see a speedup of over 1000x. Whether using generators leads to code that is faster or slower depends on the circumstances, and you should (1) always profile rather than guesses; and (2) worry about code clarify first, and performance second - if ever.

## Comparison and Logical Operators

### Comparisons

When testing for equality we use `==`.

In [35]:
x = 1

1

In [36]:
x == 2

false

For "not equal", use `!=` or `≠ (\ne<TAB>)`.

In [37]:
x != 3

true

In [38]:
x ≠ 3

true

Julia can also test approximate equality with `≈ (\approx<TAB>)`.

In [45]:
1 + 1E-8 ≈ 1

true

Be careful when using this, however, as there are subtleties involving the scales of the quantities compared.

### Combining Expressions

Here are standard logical connectives (conjunction, disjunction).

In [ ]:
true && false # and

false

In [ ]:
true || false # or

true

Remember
- `P && Q` is `true` if both are `true`, otherwise it's `false`.
- `P || Q` is `false` if both are `false`, otherwise it's `true`.